In [111]:
import urllib.request
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime

from collections import namedtuple
import pickle
import pandas as pd

from IPython.display import clear_output
from selenium import webdriver
from time import sleep
import json
from math import radians, cos, sin, asin, sqrt
import copy

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import time

In [112]:
atm = pickle.load(open('atms_target_1710.dat','rb'))

atm = ROS_ATMS

atm.head()

atm.columns  = ['atm_acqbin','atm_caid','atm_terminal','full_address','traget', 'log_traget']

atm['address'] = atm.full_address.apply(lambda x: x[:23].strip())

atm['city'] = atm.full_address.apply(lambda x: x[23:].strip()).apply(lambda x: x[:-2] if x.endswith(' G') else x)

def transform(x):
    dictionary = {"MOSKVA": "MOSCOW",
                  "SANKT-PETERBU": "SAINT PETERSBURG",
                  'S.-PETERBURG':  "SAINT PETERSBURG",
                  'ST-PETERSBURG': "SAINT PETERSBURG",
                  'ST PETERSBURG': "SAINT PETERSBURG"}
    for key, value in dictionary.items():
        x = x.replace(key, value)
    return x

atm.city = atm.city.map(transform)

atm['address_rus'] = ''
atm['lat'] = np.nan
atm['long'] = np.nan

to_replace = {
     'AV.': ' PROSPECT ',
     'AVE': ' PROSPECT ', 
     'PR-KT': ' PROSPECT ',
     'PR-T': ' PROSPECT ',
     'STR.': ' ULITSA ',
     ' ST ': ' ULITSA ',
     'UL.' : ' ULITSA ',
     'H/W.': ' SHOSSE ',
     ', LIT. ': '',
     'D.': '',
     'VLAD. ':'',
     ', KORP. ': '/',
     ' DZERZHINSK':  ' DZERZHINSKOGO ',
     '.': '. ',
     ',': ', ',
}

for i in atm.index:
    for key, value in to_replace.items():
        atm.at[i,'address'] =  atm.at[i,'address'].replace(key, value)

for i in atm.index:
    atm.at[i,'address'] = atm.at[i,'address'][4:] if atm.at[i,'address'][:4] == 'ABB ' else atm.at[i,'address']

len(atm)

# Парсер открытой информации из HeadHanter

def log_progress(sequence, every=10):
    for index, item in enumerate(sequence):
        if index % every == 0:
            clear_output()
            print(index)
        yield item

def parsing(atm, start=0):
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
    for i, atmi in log_progress(list(atm.iterrows())[start:]):
        urli = url + ' '.join([atmi.address, atmi.city]).replace(' ', '%20')
        driver.get(urli)
        try: 
            element = driver.find_element_by_xpath("//div[@class='clipboard__action-wrapper _inline']")
            atm.at[i, 'lat'],  atm.at[i, 'long'] = list(map(float, element.text.split(',')))

            addr_part1 = driver.find_element_by_xpath("//div[@class='card-title-view__title']")
            addr_part2 = driver.find_element_by_xpath("//div[@class='toponym-card-view__description']")
            atm.at[i, 'adress'] = addr_part1.text + ', ' + addr_part2.text 
        except:
            pass
        time.sleep(0.2)

parsing(atm)

In [ ]:
pickle.dump(atm, open('atm_target.data','wb'))

In [49]:
# Файл с адресами банкоматов
ros_atms = pd.read_excel('export_atms.xlsx')

In [50]:
# Переезд
travel = pd.read_excel('travel.xlsx')

In [51]:
travel.index = travel.end

In [52]:
ATMS = pickle.load(open('atms_target_1710.dat','rb'))

In [40]:
ROS_ATMS = ATMS[ATMS.caid.isin(['ROSBANK'])]

In [41]:
to_join = ros_atms[['ID Банкомата','Адрес строкой RU']]

In [42]:
to_join.columns = ['terminal', 'address_rus']

In [44]:
ROS_ATMS = pd.merge(ROS_ATMS, to_join, on=['terminal'], how='left')

In [139]:
ROS_ATMS.rename(columns={'address_rus':'address_rus_ros'})

acqbin     caid  terminal                        cardacceptorname  \
0     60312717  ROSBANK  ATM50042  Vakhrusheva, 13        Artem             
1     60312717  ROSBANK  ATM50044  LENINA, 156            Kyren             
2     60312717  ROSBANK  ATM50045  TUPOLEVA, 4            Ulan-Ude          
3     60312717  ROSBANK  ATM50047  Novocheremushkinskaya, Moskva            
4     60312717  ROSBANK  ATM50048  Oktyabrskaya, 2        Zaigrayevo        
5     60312717  ROSBANK  ATM50051  Leninskiy pr, 12       Norilsk           
6     60312717  ROSBANK  ATM50058  PR. LENINSKIY, 39      Norilsk           
7     60312717  ROSBANK  ATM50073  ZHYKOVA, 18            Abakan            
8     60312717  ROSBANK  ATM50077  ELEKTROZAVODSKAYA, 15  Sarapul           
9     60312717  ROSBANK  ATM50080  Sosnovaya Alleya, 601  Zelenograd        
10    60312717  ROSBANK  ATM50083  9 MAYA, 72             Krasnoyarsk       
11    60312717  ROSBANK  ATM50087  2 SYROMYAT. PR,1       Moskva            
12    60312717  ROSBANK  ATM50088  VAVILOVA, 1,39         Krasnoyarsk       
13    60312717  ROSBANK  ATM50090  Zavodskaya, 58         Berezovka         
14    60312717  ROSBANK  ATM50091  PR. NARODOV, 50        Abakan            
15    60312717  ROSBANK  ATM50115  Lenina pr, 7           Yakutsk           
16    60312717  ROSBANK  ATM50117  Mukhina,114            Blagoveshchensk   
17    60312717  ROSBANK  ATM50118  PYATNITSKAYA, 2,38,2   Moskva            
18    60312717  ROSBANK  ATM50126  SOVETSKAYA, 139A       Irkutsk           
19    60312717  ROSBANK  ATM50127  Krasnoarmeyskay,233    Kyzyl             
20    60312717  ROSBANK  ATM50128  KECHIL OOL, 7B         Kyzyl             
21    60312717  ROSBANK  ATM50130  KECHIL OOLA, 2B        Kyzyl             
22    60312717  ROSBANK  ATM50139  FADEEVA, 1             Vladivostok       
23    60312717  ROSBANK  ATM50144  KOCHETOVA, 1           Kyzyl             
24    60312717  ROSBANK  ATM50146  MAYAKOVSKOGO, 5A       Smolyaninovo      
25    60312717  ROSBANK  ATM50150  SOVETSKAYA PL.,5       Nizhniy Novgoro   
26    60312717  ROSBANK  ATM50151  LENINSKAYA, 64         Slavyanka         
27    60312717  ROSBANK  ATM50152  KIROVA, 48             Artem             
28    60312717  ROSBANK  ATM50164  BORISENKO, 35          Vladivostok       
29    60312717  ROSBANK  ATM50165  SOVETSKAYA, 100        Romny             
...        ...      ...       ...                                     ...   
1626  60312717  ROSBANK  ATM71883  Kirova, 63             Zabrody s.        
1627  60312717  ROSBANK  ATM71885  KARLA MARKSA, 172      Magnitogorsk      
1628  60312717  ROSBANK  ATM71886  ALTYF. SH. 22          Moskva            
1629  60312717  ROSBANK  ATM71887  SOVETSKAYA, 10         Zyryanskoye       
1630  60312717  ROSBANK  ATM71888  LENINA, 8              Kozhevnikovo      
1631  60312717  ROSBANK  ATM71889  SVOBODNYY, 82          Krasnoyarsk       
1632  60312717  ROSBANK  ATM71896  LENINA, 220            YUzhno-Sakhalin   
1633  60312717  ROSBANK  ATM71899  OMSOMOLSK.,247B        YUzhno-Sakhalin   
1634  60312717  ROSBANK  ATM71900  60 LET OKTYABRYA,46    Krasnoyarsk       
1635  60312717  ROSBANK  ATM71907  MKRN. SOLNECHNYY, 11A  Topki             
1636  60312717  ROSBANK  ATM71908  Kominterna, 139        Nizhniy Novgoro   
1637  60312717  ROSBANK  ATM71909  im V.I.Lenina pr, 54b  Volgograd         
1638  60312717  ROSBANK  ATM71910  STROITELEY, 43         Novokuznetsk      
1639  60312717  ROSBANK  ATM71914  Ilicha, 23             Inskoy            
1640  60312717  ROSBANK  ATM71915  Rublevskoye sh, 62     Moskva            
1641  60312717  ROSBANK  ATM71916  PR. LENINA, 59A        Kemerovo          
1642  60312717  ROSBANK  ATM71917  Nekrasova, 31          Abakan            
1643  60312717  ROSBANK  ATM71919  LENINA, 20B            Danilov           
1644  60312717  ROSBANK  ATM71921  EFANOVA, 22            Zyryanskoye       
1645  60312717  ROSBANK  ATM71924  LENINSKAY, 37          Pervomayskoye     
1646

In [85]:
len(ROS_ATMS)

1656

In [132]:
NOT_ROS_ATMS = ATMS[~ATMS.caid.isin(['ROSBANK'])]

In [133]:
# загрузка распарсенных данных
atms = pickle.load(open('atm_target_no_rba.data','rb'))

In [134]:
NOT_ROS_ATMS.columns = ['atm_acqbin','atm_caid','atm_terminal','full_address','traget','log_traget']

In [135]:
result = pd.merge(NOT_ROS_ATMS, atms[['atm_acqbin','atm_terminal', 'address','city','address_rus','lat','long']], on=['atm_acqbin','atm_terminal'], how='left')

In [136]:
result

atm_acqbin atm_caid atm_terminal  \
0            176      100     ATM01222   
1            176      100     ATM01392   
2            176      100     ATM01410   
3            176      155     ATM00013   
4            176      155     ATM00015   
5            176      155     ATM00019   
6            176      155     ATM00020   
7            176      155     ATM00021   
8            176      155     ATM00023   
9            176      155     ATM00024   
10           176      155     ATM00312   
11           176      155     ATM03060   
12           176      155     ATM04674   
13           176      155     ATM07097   
14           176      166     ATM00084   
15           176      166     ATM00137   
16           176      166     ATM00183   
17           176      166     ATM00637   
18           176      166     ATM01777   
19           176      166     ATM02008   
20           176      166     ATM02235   
21           176      166     ATM02497   
22           176      166     ATM04354   
23           176      166     ATM04604   
24           176      166     ATM04666   
25           176      166     ATM04690   
26           176      166     ATM06497   
27           176      166     ATM06499   
28           176      166     ATM06500   
29           176      176     ATM00321   
...          ...      ...          ...   
10748     471461     VB24       399112   
10749     471461     VB24       399121   
10750     471461     VB24       399123   
10751     471461     VB24       399124   
10752     471461     VB24       399125   
10753     471461     VB24       399132   
10754     471461     VB24       399138   
10755     471461     VB24       399146   
10756     471461     VB24       399147   
10757     471461     VB24       399150   
10758     471461     VB24       399159   
10759     471461     VB24       399161   
10760     471461     VB24       399162   
10761     471461     VB24       399164   
10762     471461     VB24       399165   
10763     471461     VB24       399166   
10764     471461     VB24       399167   
10765     471461     VB24       399169   
10766     471461     VB24       399170   
10767     471461     VB24       399173   
10768     471461     VB24       399174   
10769     471461     VB24       399176   
10770     471461     VB24       399177   
10771     471461     VB24       399179   
10772     471461     VB24       399180   
10773     471461     VB24       399182   
10774     471461     VB24       399183   
10775     471461     VB24       399185   
10776     471461     VB24       399186   
10777     471461     VB24       399192   

                                 full_address        traget  log_traget  \
0      MARATA, 17             S.PETERBURG      7.236100e+08   20.399763   
1      TYUSHINA, 11           ST.PETERSBURG    4.617000e+08   19.950426   
2      MARATA,17              SP.PETERBURG     3.279600e+08   19.608402   
3      ALTAYSKIY PER., 15     Y-SAKHALINSK     2.793000e+07   17.145212   
4      EMELYANOVA,34          Y-SAKHALINSK     2.723800e+08   19.422709   
5      AEROVOKZAL, 1          Y-SAKHALINSK     1.429900e+08   18.778285   
6      ESENINA, 15 A          Y-SAKHALINSK     1.060360e+09   20.781874   
7      PORTOVAYA, 10          KORSAKOV         3.395200e+08   19.643043   
8      SAKHALINSKAYA, 69      Y. SAKHALINSK    1.754910e+09   21.285683   
9      YEMELYANOVA, 19        YUZHNO-SAKHAL    3.733900e+08   19.738134   
10     POGRANICHNAYA,55       YUZHNO SAKHAL    4.858000e+08   20.001308   
11     KOMSOMOLSKAYA,259B     Y.SAKHALINSK     3.864000e+08   19.772384   
12     KOMMUN. PR., 32        YUZHNO SAKHAL    2.623200e+08   19.385076   
13     LENINA, 198            YUZHNO SAKHAL    1.555200e+08   18.862285   
14     KASHIRSKIY PROEZD, 13  MOSCOW           2.901000e+07   17.183151   
15     IZMAYLOVSKOE SH., 71.2BMOSCOW           2.340000e+07   16.968247   
16     KASHIRSKOE SH., 26     MOSCOW           2.381500e+08   19.288411   
17     ENISEYSKAYA, 19.1      MOSCOW           4.78

In [90]:
NOT_ROS_ATMS

atm_acqbin atm_caid atm_terminal  \
0            176      100     ATM01222   
1            176      100     ATM01392   
2            176      100     ATM01410   
3            176      155     ATM00013   
4            176      155     ATM00015   
5            176      155     ATM00019   
6            176      155     ATM00020   
7            176      155     ATM00021   
8            176      155     ATM00023   
9            176      155     ATM00024   
10           176      155     ATM00312   
11           176      155     ATM03060   
12           176      155     ATM04674   
13           176      155     ATM07097   
14           176      166     ATM00084   
15           176      166     ATM00137   
16           176      166     ATM00183   
17           176      166     ATM00637   
18           176      166     ATM01777   
19           176      166     ATM02008   
20           176      166     ATM02235   
21           176      166     ATM02497   
22           176      166     ATM04354   
23           176      166     ATM04604   
24           176      166     ATM04666   
25           176      166     ATM04690   
26           176      166     ATM06497   
27           176      166     ATM06499   
28           176      166     ATM06500   
29           176      176     ATM00321   
...          ...      ...          ...   
11153     471461     VB24       399112   
11154     471461     VB24       399121   
11155     471461     VB24       399123   
11156     471461     VB24       399124   
11157     471461     VB24       399125   
11158     471461     VB24       399132   
11159     471461     VB24       399138   
11160     471461     VB24       399146   
11161     471461     VB24       399147   
11162     471461     VB24       399150   
11163     471461     VB24       399159   
11164     471461     VB24       399161   
11165     471461     VB24       399162   
11166     471461     VB24       399164   
11167     471461     VB24       399165   
11168     471461     VB24       399166   
11169     471461     VB24       399167   
11170     471461     VB24       399169   
11171     471461     VB24       399170   
11172     471461     VB24       399173   
11173     471461     VB24       399174   
11174     471461     VB24       399176   
11175     471461     VB24       399177   
11176     471461     VB24       399179   
11177     471461     VB24       399180   
11178     471461     VB24       399182   
11179     471461     VB24       399183   
11180     471461     VB24       399185   
11181     471461     VB24       399186   
11182     471461     VB24       399192   

                                 full_address        traget  log_traget  
0      MARATA, 17             S.PETERBURG      7.236100e+08   20.399763  
1      TYUSHINA, 11           ST.PETERSBURG    4.617000e+08   19.950426  
2      MARATA,17              SP.PETERBURG     3.279600e+08   19.608402  
3      ALTAYSKIY PER., 15     Y-SAKHALINSK     2.793000e+07   17.145212  
4      EMELYANOVA,34          Y-SAKHALINSK     2.723800e+08   19.422709  
5      AEROVOKZAL, 1          Y-SAKHALINSK     1.429900e+08   18.778285  
6      ESENINA, 15 A          Y-SAKHALINSK     1.060360e+09   20.781874  
7      PORTOVAYA, 10          KORSAKOV         3.395200e+08   19.643043  
8      SAKHALINSKAYA, 69      Y. SAKHALINSK    1.754910e+09   21.285683  
9      YEMELYANOVA, 19        YUZHNO-SAKHAL    3.733900e+08   19.738134  
10     POGRANICHNAYA,55       YUZHNO SAKHAL    4.858000e+08   20.001308  
11     KOMSOMOLSKAYA,259B     Y.SAKHALINSK     3.864000e+08   19.772384  
12     KOMMUN. PR., 32        YUZHNO SAKHAL    2.623200e+08   19.385076  
13     LENINA, 198            YUZHNO SAKHAL    1.555200e+08   18.862285  
14     KASHIRSKIY PROEZD, 13  MOSCOW           2.901000e+07   17.183151  
15     IZMAYLOVSKOE SH., 71.2BMOSCOW           2.340000e+07   16.968247  
16     KASHIRSKOE SH., 26     MOSCOW           2.381500e+08   19.288411  
17     ENISEYSKAYA, 19.1      MOSCOW           4.780000e+07   17.6825

In [87]:
result[result.lat.isnull()]

atm_acqbin atm_terminal atm_caid  \
0            176     ATM01222      NaN   
1            176     ATM01392      NaN   
2            176     ATM01410      NaN   
3            176     ATM00013      NaN   
4            176     ATM00015      NaN   
5            176     ATM00019      NaN   
6            176     ATM00020      NaN   
7            176     ATM00021      NaN   
8            176     ATM00023      NaN   
9            176     ATM00024      NaN   
10           176     ATM00312      NaN   
11           176     ATM03060      NaN   
12           176     ATM04674      NaN   
13           176     ATM07097      NaN   
14           176     ATM00084      NaN   
15           176     ATM00137      NaN   
16           176     ATM00183      NaN   
17           176     ATM00637      NaN   
18           176     ATM01777      NaN   
19           176     ATM02008      NaN   
20           176     ATM02235      NaN   
21           176     ATM02497      NaN   
22           176     ATM04354      NaN   
23           176     ATM04604      NaN   
24           176     ATM04666      NaN   
25           176     ATM04690      NaN   
26           176     ATM06497      NaN   
27           176     ATM06499      NaN   
28           176     ATM06500      NaN   
29           176     ATM00321      NaN   
...          ...          ...      ...   
10647     471461       398907     VB24   
10659     471461       398929     VB24   
10666     471461       398937     VB24   
10667     471461       398938     VB24   
10668     471461       398939     VB24   
10670     471461       398943     VB24   
10679     471461       398963     VB24   
10680     471461       398964     VB24   
10682     471461       398968     VB24   
10687     471461       398998     VB24   
10694     471461       399011     VB24   
10702     471461       399023     VB24   
10706     471461       399032     VB24   
10709     471461       399037     VB24   
10715     471461       399045     VB24   
10716     471461       399046     VB24   
10718     471461       399048     VB24   
10728     471461       399063     VB24   
10731     471461       399068     VB24   
10737     471461       399079     VB24   
10738     471461       399080     VB24   
10749     471461       399121     VB24   
10757     471461       399150     VB24   
10758     471461       399159      NaN   
10763     471461       399166      NaN   
10767     471461       399173     VB24   
10771     471461       399179     VB24   
10773     471461       399182      NaN   
10774     471461       399183     VB24   
10777     471461       399192     VB24   

                                 full_address       traget  log_traget  \
0                                         NaN          NaN         NaN   
1                                         NaN          NaN         NaN   
2                                         NaN          NaN         NaN   
3                                         NaN          NaN         NaN   
4                                         NaN          NaN         NaN   
5                                         NaN          NaN         NaN   
6                                         NaN          NaN         NaN   
7                                         NaN          NaN         NaN   
8                                         NaN          NaN         NaN   
9                                         NaN          NaN         NaN   
10                                        NaN          NaN         NaN   
11                                        NaN          NaN         NaN   
12                                        NaN          NaN         NaN   
13                                        NaN          NaN         NaN   
14                                        NaN          NaN         NaN   
15                                        NaN          NaN         NaN   
16                                        NaN          NaN         NaN   
17                                        NaN          NaN         Na

In [86]:
1656 + 7550

9206

In [151]:
rba = atms[atms.full_address.apply(lambda x: x.startswith('RBA'))]

In [152]:
rba['id'] = rba.address.apply(lambda x: ''.join(x.split()[1:]))